In [2]:
MAX_AREA = 500.0
CROPS = [:wheat, :corn, :sugar_beet]
PLANTING_COST = Dict(:wheat => 150.0, :corn => 230.0, :sugar_beet => 260.0)
MIN_QUANTITIES = Dict(:wheat => 200.0, :corn => 240.0, :sugar_beet => 0.0)
QUOTA_MAX = Dict(:wheat => Inf, :corn => Inf, :sugar_beet => 6_000.0)
SELL_IN_QUOTA = Dict(:wheat => 170.0, :corn => 150.0, :sugar_beet => 36.0)
SELL_NO_QUOTA = Dict(:wheat => 0.0, :corn => 0.0, :sugar_beet => 10.0)
BUY_PRICE = Dict(:wheat => 238.0, :corn => 210.0, :sugar_beet => 1_000.0)
MEAN_YIELD = Dict(:wheat => 2.5, :corn => 3.0, :sugar_beet => 20.0)
YIELD_MULTIPLIER = Dict(:good => 1.2, :fair => 1.0, :bad => 0.8)

Dict{Symbol, Float64} with 3 entries:
  :bad  => 0.8
  :good => 1.2
  :fair => 1.0

In [3]:
using SDDP, HiGHS, Statistics

In [4]:
function add_state_variables(subproblem)
    @variable(subproblem, area[c = CROPS] >= 0, SDDP.State, initial_value = 0)
end

add_state_variables (generic function with 1 method)

In [5]:
function create_first_stage_problem(subproblem)
    @constraint(
        subproblem,
        sum(subproblem[:area][c].out for c in CROPS) <= MAX_AREA
    )
    @stageobjective(
        subproblem,
        -sum(PLANTING_COST[c] * subproblem[:area][c].out for c in CROPS)
    )
end

create_first_stage_problem (generic function with 1 method)

In [6]:
function second_stage_variables(subproblem)
    @variables(subproblem, begin
        0 <= yield[c = CROPS]                          # tonnes/acre
        0 <= buy[c = CROPS]                            # tonnes
        0 <= sell_in_quota[c = CROPS] <= QUOTA_MAX[c]  # tonnes
        0 <= sell_no_quota[c = CROPS]                  # tonnes
    end)
end

second_stage_variables (generic function with 1 method)

In [7]:
function second_stage_constraint_min_quantity(subproblem)
    @constraint(
        subproblem,
        [c = CROPS],
        subproblem[:yield][c] + subproblem[:buy][c] -
        subproblem[:sell_in_quota][c] - subproblem[:sell_no_quota][c] >=
        MIN_QUANTITIES[c]
    )
end

second_stage_constraint_min_quantity (generic function with 1 method)

In [8]:
function second_stage_objective(subproblem)
    @stageobjective(
        subproblem,
        sum(
            SELL_IN_QUOTA[c] * subproblem[:sell_in_quota][c] +
            SELL_NO_QUOTA[c] * subproblem[:sell_no_quota][c] -
            BUY_PRICE[c] * subproblem[:buy][c] for c in CROPS
        )
    )
end

second_stage_objective (generic function with 1 method)

In [9]:
function second_stage_uncertainty(subproblem)
    @constraint(
        subproblem,
        uncertainty[c = CROPS],
        1.0 * subproblem[:area][c].in == subproblem[:yield][c]
    )
    SDDP.parameterize(subproblem, [:good, :fair, :bad]) do ω
        for c in CROPS
            JuMP.set_normalized_coefficient(
                uncertainty[c],
                subproblem[:area][c].in,
                MEAN_YIELD[c] * YIELD_MULTIPLIER[ω],
            )
        end
    end
end

second_stage_uncertainty (generic function with 1 method)

In [10]:
model = SDDP.LinearPolicyGraph(
    stages = 2,
    sense = :Max,
    upper_bound = 500_000.0,
    optimizer = HiGHS.Optimizer,
    direct_mode = false,
) do subproblem, stage
    add_state_variables(subproblem)
    if stage == 1
        create_first_stage_problem(subproblem)
    else
        second_stage_variables(subproblem)
        second_stage_constraint_min_quantity(subproblem)
        second_stage_uncertainty(subproblem)
        second_stage_objective(subproblem)
    end
end

A policy graph with 2 nodes.
 Node indices: 1, 2


In [11]:
SDDP.train(model; iteration_limit = 20)

------------------------------------------------------------------------------
          SDDP.jl (c) Oscar Dowson and SDDP.jl contributors, 2017-23

Problem
  Nodes           : 2
  State variables : 3
  Scenarios       : 3.00000e+00
  Existing cuts   : false
  Subproblem structure                      : (min, max)
    Variables                               : (7, 19)
    VariableRef in MOI.LessThan{Float64}    : (1, 2)
    AffExpr in MOI.LessThan{Float64}        : (1, 1)
    AffExpr in MOI.GreaterThan{Float64}     : (3, 3)
    AffExpr in MOI.EqualTo{Float64}         : (3, 3)
    VariableRef in MOI.GreaterThan{Float64} : (3, 16)
Options
  Solver          : serial mode
  Risk measure    : SDDP.Expectation()
  Sampling scheme : SDDP.InSampleMonteCarlo

Numerical stability report
  Non-zero Matrix range     [1e+00, 2e+01]
  Non-zero Objective range  [1e+00, 1e+03]
  Non-zero Bounds range     [6e+03, 5e+05]
  Non-zero RHS range        [2e+02, 5e+02]
No problems detected

 Iteration    Simul

In [21]:
#@assert 
isapprox(SDDP.calculate_bound(model), 108_390.0, atol = 0.1)

true

In [18]:
sims = SDDP.simulate(model, 100, [:area])
sims[100][2][:area]
#mu = round(mean([s[1][:area]:wheat][1] for s in sims]), digits = 2)

1-dimensional DenseAxisArray{SDDP.State{Float64},1,...} with index sets:
    Dimension 1, [:wheat, :corn, :sugar_beet]
And data, a 3-element Vector{SDDP.State{Float64}}:
 SDDP.State{Float64}(169.9999999999995, 0.0)
 SDDP.State{Float64}(80.00000000000024, 0.0)
 SDDP.State{Float64}(250.00000000000028, 0.0)